# Economic Experiment

Add description of the experiment here

## Import Statements

In [ ]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore, Likelihood, ProbabilityDifference, ProbabilityPointer, FavouriteProbability
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.evaluators.base_evaluators import BettingActivity
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError

import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import itertools as it

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## External actors

We create a bookmaker and a betting strategy that are going to interact with the generated networks.

In [ ]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)

In [ ]:
betting = FixedBetting(100)

We also create an error rating that we are going to use for the calculated forecasts

In [ ]:
rating_error = RatingFunctionError(error='normal', loc=0, scale=50)
rating_error_bookmaker = RatingFunctionError(error='normal', loc=0, scale=25)

## Generation of networks

Configure the number of leagues for the experiment:

In [ ]:
numberLeagues = 40

For every league, we generate the schedule of matches and true dimensions and we add odds for each game.

In [ ]:
networks = []
gen_start_time = time.time()
for league in range(numberLeagues):
    print(f"League : {league}")
    network: BaseNetwork = factory.new_network(
        'multiple-round-robin',
        teams=20,
        days_between_rounds=7,
        seasons=10,
        league_teams=20,
        league_promotion=0,
        create=True,
        true_forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients = [-0.9,0.3], 
            beta_parameter=0.006
        ),
        true_rating=ControlledTrendRating(
            starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
            delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
            trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
            season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10)
        )
    )
    #adding biased bookmaker forecast to the network
    network.add_forecast(
        forecast=LogFunctionForecast(
            outcomes=['home', 'draw', 'away'], 
            coefficients=[-0.8, 0.2], 
            beta_parameter=0.006, 
            home_team_error=rating_error_bookmaker, 
            away_team_error=rating_error_bookmaker
        ),
        forecast_name='bookmaker_forecast',
        base_ranking='true_rating'
    )
    #adding odds    
    network.add_odds(
        bookmaker_name="bm",
        bookmaker=bookmaker,
        base_forecast='bookmaker_forecast'
    )

    networks.append(network)
print(f"{numberLeagues} leagues added in {float(time.time() - gen_start_time)} seconds.")

## Main loop

We iterate over the grid of possible parameters for the experiment and aggregate the final results.

In [ ]:
def aggregate_measures(list_of_matches, prefix='', addBettingMetrics=True):
    if addBettingMetrics:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches]),
            prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
            prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches]),
            prefix+'_betting_activity': sum([m[3]['metrics']['betting_activity']['qty'] for m in list_of_matches])
        }
    else:
        return {
            prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]),
            prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]),
            prefix+'_prob_diff': np.mean([m[3]['metrics']['prob_diff'] for m in list_of_matches]),
            prefix+'_draw_prob': np.mean([m[3]['metrics']['draw_prob'] for m in list_of_matches]),
            prefix+'_fav_prob': np.mean([m[3]['metrics']['fav_prob'] for m in list_of_matches])
        }

Getting first the values for the true model and the bookmaker model. Remove some of the if they are not needed and if the IS/OOS split is not required.

In [ ]:
entire_model_values = {}
for forecast_pointer in ['true_forecast', 'bookmaker_forecast']:
    all_matches =  []
    is_matches =  []
    oos_matches =  []
    for network in tqdm(networks):
        rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer, probability_index=1)
        fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
        network.add_evaluation([
            (rps, 'rps'),
            (likelihood, 'likelihood'),
            (difference, 'prob_diff'),
            (draw_probability, 'draw_prob'),
            (fav_probability, 'fav_prob')
        ])
        all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
        is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 6]
        oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 5]

    result_all = aggregate_measures(all_matches, f'all_{forecast_pointer}', False)
    result_is = aggregate_measures(is_matches, f'is_{forecast_pointer}', False)
    result_oos = aggregate_measures(oos_matches, f'oos_{forecast_pointer}', False)
    entire_model_values = {
        **entire_model_values,
        **result_is, **result_oos, **result_all
    }
print(entire_model_values)

Configuration for splits:

In [ ]:
splits = 4

In [ ]:
def cross_validate_betting_returns(matches_array, n_splits, model_hint):
    kf = np.array_split(matches_array, n_splits)
    combinations = [(in_s, out_s) for in_s, out_s in it.permutations(range(len(kf)), r=2)]
    result = []
    for in_sample_index, out_sample_index in combinations:
        result_is = aggregate_measures(kf[in_sample_index], "is")
        result_oos = aggregate_measures(kf[out_sample_index], "oos")
        result.append({
            "permutation": f"{in_sample_index} x {out_sample_index}",
            "model_hint": model_hint,
            **result_is,
            **result_oos
        })
    return result

In [ ]:
result_list = []
experiment_start_time = time.time()
cv_result = []
for c0 in np.arange(-1.20, -0.60, 0.1):
    for c1 in np.arange(0.0, 0.60, 0.1):
        for beta in np.arange(0.002, 0.010, 0.002):
            print(f'Variables: c0: {c0}, c1: {c1} and beta: {beta}')
            forecast_pointer = f"player_forecast_{c0:.2f}_{c1:.2f}_{beta:.3f}"
            cell_start_time = time.time()
            all_matches =  []
            is_matches =  []
            oos_matches =  []
            
            for network in networks:
                rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
                betting_activity = BettingActivity(outcomes=['home', 'draw', 'away'], player_name = 'b')
                likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                difference = ProbabilityDifference(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
                draw_probability = ProbabilityPointer(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer, probability_index=1)
                fav_probability = FavouriteProbability(outcomes=['home', 'draw', 'away'], forecast_name = forecast_pointer)
            
                network.add_forecast(
                    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
                    forecast_name=forecast_pointer,
                    base_ranking='true_rating'
                )
                network.add_bets(
                    bettor_name='b',
                    bookmaker='bm',
                    betting=betting,
                    base_forecast=forecast_pointer
                )
                network.add_evaluation([
                    (rps, 'rps'),
                    (betting_returns, 'betting_returns'),
                    (betting_activity, 'betting_activity'),
                    (likelihood, 'likelihood'),
                    (difference, 'prob_diff'),
                    (draw_probability, 'draw_prob'),
                    (fav_probability, 'fav_prob')
                ])
                
                all_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()]
                is_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 6]
                oos_matches += [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 5]
            cv_result += cross_validate_betting_returns(np.array(all_matches), splits, forecast_pointer)
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all, **entire_model_values
            }
            result_list.append(result)
            print(f"Finished in {float(time.time() - cell_start_time)} seconds")

print(f"Experiment finished in {float(time.time() - experiment_start_time)} seconds with {len(result_list)} observations.")

df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")
            

In [ ]:
cv_df = pd.DataFrame(cv_result)

In [ ]:
cv_df[cv_df['permutation'] == "0 x 1"].sort_values(by='is_rps').head(144)

In [ ]:
# Optimizing for RPS
by_rps_df = cv_df.loc[cv_df.groupby('permutation').is_rps.idxmin(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')
# Optimizing for likelihood
by_likelihood_df = cv_df.loc[cv_df.groupby('permutation').is_likelihood.idxmax(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')
# Optimizing for betting
by_betting_df = cv_df.loc[cv_df.groupby('permutation').is_betting_returns.idxmax(), ['permutation', 'model_hint', 'oos_betting_returns',	'oos_betting_returns_expected']].set_index('permutation')

In [ ]:
by_betting_df.dtypes

In [ ]:
cv_study_df = by_rps_df.join(
    by_likelihood_df.join(
        by_betting_df,
        lsuffix='',
        rsuffix='_betting',
    ),
    lsuffix='_rps',
    rsuffix='_likelihood',
)

In [44]:
cv_study_df.to_excel("Permutations.xlsx")

,model_hint_rps,oos_betting_returns_rps,oos_betting_returns_expected_rps,model_hint_likelihood,oos_betting_returns_likelihood,oos_betting_returns_expected_likelihood,model_hint_betting,oos_betting_returns_betting,oos_betting_returns_expected_betting
permutation,,,,,,,,,
0 x 1,player_forecast_-0.80_0.40_0.006,217.88908,148.31875,player_forecast_-0.80_0.40_0.006,217.88908,148.31875,player_forecast_-1.10_0.50_0.008,106.78266,102.52984
0 x 2,player_forecast_-0.80_0.40_0.006,29.20471,144.66903,player_forecast_-0.80_0.40_0.006,29.20471,144.66903,player_forecast_-1.10_0.50_0.008,50.16522,108.43502
0 x 3,player_forecast_-0.80_0.40_0.006,270.30734,144.76062,player_forecast_-0.80_0.40_0.006,270.30734,144.76062,player_forecast_-1.10_0.50_0.008,133.22385,105.88461
1 x 0,player_forecast_-0.80_0.40_0.006,22.17866,147.89816,player_forecast_-0.80_0.40_0.006,22.17866,147.89816,player_forecast_-1.00_0.20_0.006,60.17866,144.26993
1 x 2,player_forecast_-0.80_0.40_0.006,29.20471,144.66903,player_forecast_-0.80_0.40_0.006,29.20471,144.66903,player_forecast_-1.00_0.20_0.006,30.36982,140.23863
1 x 3,player_forecast_-0.80_0.40_0.006,270.30734,144.76062,player_forecast_-0.80_0.40_0.006,270.30734,144.76062,player_forecast_-1.00_0.20_0.006,215.99303,142.50920
2 x 0,player_forecast_-0.80_0.30_0.006,-43.43703,66.12146,player_forecast_-0.80_0.30_0.006,-43.43703,66.12146,player_forecast_-1.20_0.30_0.008,128.55569,93.69572
2 x 1,player_forecast_-0.80_0.30_0.006,194.87611,66.14607,player_forecast_-0.80_0.30_0.006,194.87611,66.14607,player_forecast_-1.20_0.30_0.008,74.69295,90.80741
2 x 3,player_forecast_-0.80_0.30_0.006,75.61714,61.83845,player_forecast_-0.80_0.30_0.006,75.61714,61.83845,player_forecast_-1.20_0.30_0.008,133.08318,92.93767


In [ ]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])